## **灰色模型Python编程指南**

#### **一、灰色模型常用的数据处理方法**

灰色模型是较为简单的一种预测模型，其计算过程通常都较为简单。本文主要简单介绍灰色模型常用的处理方法和计算过程，尝试将其建模过程模块化，从而方便我们进一步利用Python语言和面向对象的思想进行实现。



灰色模型的一般建模过程

``` mermaid
graph LR
a[累加处理]
b[参数估计] 
c[时间响应]
d[还原处理]

a-->b
b-->c
c-->d

```

- **累加与还原**：包括整数阶、分数阶累加，新信息优先累加和其它累加，这里所有累加与还原全部配套，必须严格满足互为逆运算
- **参数估计**: 这里需要处理的内容有点多
  - 背景值计算
  - 矩阵构造
  - 参数求解（最小二乘或其它方法）
- **时间响应**：这里属于特别个性化的地方，尤其是常用连续响应式的模型对应的公式不尽相同。但如果采用GMC(1, n)模型的统一格式则十分简单

##### **二、常用库和API**

对应上述内容，我们逐个梳理其需要用到的库及常用方法。

**1、累加与还原**

- **整数阶累加**-- `cumsum`函数：这里我们用几个简单的例子来说明。

In [26]:
import numpy as np
x = np.array([1,2,3,4,5])
x1 = x.cumsum()
print('x:',x)
print('x1:',x1)

x: [1 2 3 4 5]
x1: [ 1  3  6 10 15]


----
所以在做整数阶累加时，直接一步就可以搞定。至于还原就更简单了，还原本身就是差分，直接使用`dif`函数即可：

In [72]:
x0 = np.diff(x1)
print(x0)

[2 3 4 5]


同时也注意到，差分时第一项是被略掉的，这里可以再将第一个元素添加进去即可。这一操作可以通过`concatenate`函数来实现，这个函数在下面讲参数估计时再作介绍。

- **分数阶累加**：二项式系数、迭代器

分数阶累加相对就较为复杂，由于要用到二项式系数，因此需要用到相应的库。这时除了使用`numpy`以外也可以选择使用`scipy`中更为丰富的库。

In [22]:
from scipy.special import binom 
b = binom(5,0.5)
print(b)

2.5868993924777905


注意上述代码中第一二个变量分别是`n,k`。这里`k`可以是小数，也就是说，这里定义的二项式系数本身就是广义二项式系数，非常好用。另外仍然要提醒一句，Python这个函数只提供了$k>0$时的系数，如果$k<0$则需要手动对其进行调整。

- **提升循环效率**：迭代器

另外，分数阶累加的处理的繁杂之处不仅是这个二项式系数，更主要的目前几乎不大可能直接用矩阵实现（如果有，请告诉我。另外我们已经证明过它矩阵实现的一种方法，但经过测试非常之慢，并且有病态的风险）。而我们知道绝大多数语言（C除外）的循环都是比较慢的，那么为了提高效率，我们使用迭代器。

关于迭代器的内容比较多，这里主要是试图使用迭代器来简化我们的代码。

这里目前没有时间细讲，有兴趣的同学可以自行参考python的[迭代器]: https://www.runoob.com/python3/python3-iterator-generator.html 文档。如果有空我们单独发一个教程来讲使用迭代器实现分数阶累加。

**2、参数估计**

前面已经提到，参数估计是灰色模型中看似内容最多的地方，但在实现起来其实并不困难。总体来说基本上只需要`numpy`中的几个技巧。

- **背景值计算**：错位相加
看下面这个例子

In [28]:
import numpy as np
x = np.array([1,2,3,4,5])
x1 = x.cumsum()
z1 = 0.5*(x1[1:]+x1[:-1])

print('x:',x)
print('x1:',x1)
print('z1:',z1)

x: [1 2 3 4 5]
x1: [ 1  3  6 10 15]
z1: [ 2.   4.5  8.  12.5]


----
由于背景值的公式是：
$$
  z^{(1)}(k)=0.5\left(x^{(1)}(k)+x^{(1)}(k-1)\right)
$$

所以很自然想到其实直接对其进行错位相加即可。另外如果要对$x^{(1)}(k)$乘上系数，那就在代码中对应位置乘以系数即可。

不过对于较为复杂的背景值，可能还是只能老实用循环来处理，那么继续使用迭代器即可。

- **矩阵的构造**

在Python中使用矩阵相对Matlab而言就麻烦得多了。`numpy`中有`mat`结构，但矩阵也可以当作数组（通常是二维，也可以是三维，例如`tensor`)。这里比较方便的做法是对上述算好的内容进行拼接，这时使用函数`concatenate`。下面我们举一个简单的例子，更详细的用法参考文档https://www.runoob.com/numpy/numpy-array-manipulation.html ， 也可以参考官方文档。

In [47]:
# 接上面的变量不变

z1 = z1.reshape([-1,1])
ones_vec = np.ones([4,1]).astype(np.float)

B = np.concatenate((-z1, ones_vec), axis = 1)

print(B)

[[ -2.    1. ]
 [ -4.5   1. ]
 [ -8.    1. ]
 [-12.5   1. ]]


这里用到的技巧比较多，稍有点麻烦。
`reshape`主要是将数组变成一个4行1列的数组（注意这里仍然是数组，不是`mat`），另外`reshape`中$-1$表示先分别其它维，这一维自动生成（个人是这样理解的）；
`astype`主要是让`ones`中的数字是`float`类型，因为默认是整型；
`concatenate`中`axis`表示维度，第一维是行，第二维是列，那么`axis=1`表示按列拼接

看到这里，也容易看出，python的矩阵操作相对`matlab`的确是要麻烦很多，并且对数据类型的要求也要严格一些。

- **参数求解**：广义逆矩阵

这里稍讲一点数学。我们知道一个矩阵$A$的逆矩阵可以写成$A^{-1}$，但这仅当$A$可逆时才行得通。这里我们的$B$并不是方阵，所以自然不存在线性代数中的逆矩阵，因此要使用**广义逆**这个概念。关于概念本身我们不作延伸，知道一点即可：$(B^TB)^{-1}$ 就是$B$的广义逆。

许多初学者常在这里写许多代码来计算这个矩阵，其实完全没有必要，在`matlab`和`numpy`里面都提供了这个函数，并且都叫`pinv`。

In [39]:
print(np.linalg.pinv(B))

[[ 0.07630522  0.03614458 -0.02008032 -0.09236948]
 [ 0.76506024  0.4939759   0.11445783 -0.37349398]]


那么我们要计算参数，就非常容易了：

In [77]:
y = np.exp(0.5*x).reshape([-1,1])

B_y = np.matmul(B.T,y[1:,0]).reshape([-1,1])



params = np.matmul(np.linalg.pinv(B),y[1:,0])
a = params[0]
b = params[1]

print('a = {0}\n b = {1} '.format(a,b))


a = -0.9042573703496735
 b = 0.5891429897475113 


另外，实际上也可以将模型看作一个线性回归式，也就是将$x^{(0)}$视作$y$, 将$z^{(1)}$视作$x$，那么就可以使用线性回归的API进行计算了。线性回归的API就很多了，除了`numpy.linalg`之外，还有`statsmodel,sklearn`等等，这里不再赘述了。

 **3、响应式计算**

响应式计算通常直接采用循环就能很简单地实现，循环中直接敲入响应式的公式即可。

不过有时候我们可能需要建立许多灰色模型，比如使用`Rolling`的时候，这时也希望提升计算效率。那么这里就又需要使用矩阵（向量）计算了。

对于单变量模型而言，这一操作是非常简单的，在Python3中我们可以用这样一句代码，先生成$k=[1,2,3,4,5,...]$

In [76]:
k = np.arange(1,11)

print(k)

[ 1  2  3  4  5  6  7  8  9 10]


注意这里`arange`中是从1到11，而不是1到10（个人认为这是python比较脑残的一点。不过习惯了之后就会发现，Python的下标操作统统是“左闭右开”的形式。
**例如** 这里左端点是1，它是闭区间端点，因为可以取到1, 相反右边的11是开区间的端点，所以无法取到它，而只能取到它前一个整数。

有了这个k向量，我们就可以很容易算出$\hat{x}^{(1)}$的所有值了：

In [80]:
x1_pred = (x[0]- b/a)*np.exp(-a*k)+ b/a

print(x1_pred)

[3.42789639e+00 9.42503566e+00 2.42385506e+01 6.08293677e+01
 1.51212231e+02 3.74466658e+02 9.25926720e+02 2.28808650e+03
 5.65275310e+03 1.39638056e+04]


而计算$\hat{x}^{(0)}$时则继续使用`diff`函数即可。注意这里仍然需要重新将初值重新加回来。如果使用其它累加，则对应替换各累加即可。

上述内容将灰色模型的建模过程和对应的关键Python函数、技巧、对应库、技巧介绍了一遍。可以看到灰色模型的建模过程是非常简单的，这是由灰色模型本身的特点决定的。

另外需要注意的是，上述内容虽然可以用到所有灰色模型的实现中，但有一些模型的实现仍然是比较麻烦的，比如多变量灰色模型，Kernel-based灰色模型等等。这些内容相对繁琐，如果有时间单独来讨论。 当然，只要搞清楚矩阵计算，或者单纯去码数学公式，也是完全可以实现的。



##### **三、灰色模型的面向对象设计和封装**

在第一节中已经将灰色模型的过程进行了分解，那么如果要设计一个灰色模型的`class`，我们也可以参考这一分解内容。

这里将灰色模型基本的`class`设计写出来，后面再进一步去实现。

``` python
class BaseGreyModel:
    # 构造函数
    # 
    __init__.self(BaseGreyModel):
        
        # 基本属性
        self.params = []
    
    def fit(self, xtrain, ytrain):
        self.params = _getparams()
    
        return self
    
    def predict(self, x):
        
        
        resp = _response(x) # 或者
        
        
        
        return y_pred

    # 这里将上述构造矩阵和计算参数的过程全部封装在一起，注意几乎所有模型都是用同样的方式，所以这一封装将十分有用。由于模式完全一致，因此直接写在基类中。
    def _getparams(x,z1):
    
        return params
    
    
    # 同样，根据cum_type判断之前使用的是哪种累加，以此统一调用还原式的方式
    def _getreverse(x1, cum_type = None):
        
        return hat_x
    
    
    # 在计算响应式的时候一般只需要使用时间点和参数即可
    # 但多变量模型则相对麻烦，尤其是计算过程必须是依次序进行的，比如只提供第3、4组输入而没有第2组，就完全无法计算。
    # 因此这里暂时设定在拟合和预测的过程中，动态记录下输入值的信息，同时对输入进行扩展
    # 如果多变量模型，那么在预测时要求提供输入向量和它对应的编号，以此判断是否未按顺序进行计算，从而保证计算过程无误
    def _response(self, x):
        
        # 在模型中设定一个参数来判断是否为多变量模型，以此调整x的使用方式
        if self.is_multi:
            
            # 在这里必须要检查序列的信息在当前对象中是否完整，如果断片，至少要报警！！
            pass
        else:
            pass
        
        return resp
    
    
#  自定义常用函数库或API

# 累加生成根据不同设置选择不同生成方法，这里个人觉得完全可以写成class,更方便在greymodel里面调用

def accumulation(x, inverse=False, type=None, type_params =None):
        
        # 正、逆变换封装在一个函数中
        if inverse:
            return inerse_x
        else:
            return accumulated_x

# 类似上述内容，根据type来选择背景值公式，这里暂时只返回长度为 N-1的背景值 
def get_backgroundvalues(x1,type=None):
    
    return back_z1
```

采用上述方式定义GreyModel基类可能都还有些地方多余了，在实际开发中再考虑对其进一步细化。

利用这种方式定义，最主要的好处是在于，它已经和主流的预测建模库的模式完全一样了。这样会带来极大的便利是各种库的方法就能非常容易地集成在这个库里。

举个例子，比如我们想用`sklearn`里的`GridSeachCV`，那么对被调参的模型的要求是它必须是一个`Estimator`，更直接一点的意思是它必须要有`fit`和`predict`两种方法，而这两种方法又必须是上述代码中定义的样子。

当然我们通常不会用普通的`k-fold`来调参，但可以用`walking windows`（还有很多名字，比如expanding window等）来对它进行调参。

还有更多好玩的，比如我们可以用集成学习对灰色模型进行提升了，比如`Voting`；我们还可以考虑使用贝叶斯优化，等等等。

##### **四、小结与展望**

这个笔记简单介绍了需要用到的Python函数、技巧和相应的库，按常理说但凡有一点编程基础，尤其有MATLAB基础的同学应该可以很快上手。（尤其GM(1,1)模型实际上已经在这个笔记里实现得差不多了，不过有坑，要注意）

这个基础上，顺便对灰色模型的建模过程进行了拆解和模块化。这一拆解非常重要，因为目前主流的库里面，但凡涉及时序模型的时候它的预测过程都是需要自己去写循环才能实现的。而这个笔记里提出了一种控制序列预测机制的思路，就可以使用户在使用这些模型的时候十分方便。这一点也会极大简化程序员的工作量，更能避免设计时出错（因为自己去对下标，书写检查逻辑等都很容易出错，而且多人开发时特别容易出现混乱）。 当然这一思路一开始可能会显得比较麻烦，因为需要在外部对数据进行修改，不过我相信随着不断推进，这种方式一定可以得到完善，并且被广泛采用。（尤其NARX型的模型）

最后，整个笔记的书写目的，其实不只是为了初学灰色模型的时候能节约些时间，更重要的是想要再次强调充分利用现有包库的思想。我常说科学与技术的进步，就是在前人的基础上不断发展，就是这个意思。